In [1]:
import sys
sys.path.append("/Users/mocquin/MYLIB10/MODULES/noise3D/")

In [2]:
import numpy as np

import noise3d

# We'll be creating 3D sequences with T=100 frames, V=100 lines, and H=100 columns.
T = 100
V = 100
H = 100
# but you can define any shape

# Create some seq to apply operators on
seq = noise3d.genseq.genseq_3dnoise_seq(T, V, H, [1 for i in range(7)], [0 for i in range(7)])

Please note that all operators are linear and their composition is commutative. For eg : dt(dv(A)) = dv(dt(A)).

# Base operators dt, dv, dh

In [3]:
# These operator just apply mean on the input sequence in the direction, and keep the shape on the input sequence
print("Input seq")
print(seq.shape)
print(np.mean(seq))
print(np.std(seq))
print()

temporaly_averaged = noise3d.opr.dt(seq)

print("After temporal operator dt")
print(temporaly_averaged.shape)
print(np.mean(temporaly_averaged))
print(np.std(temporaly_averaged))
# notice the standard deviation is less than original, since some of the noise was filtered by the temporal mean operator


Input seq
(100, 100, 100)
-0.11922236369777166
2.5617260178221977

After temporal operator dt
(1, 100, 100)
-0.11922236369777173
1.6703179362708898


About dimensions and shapes : 

In [4]:
# notice the output is the same shape as input
# this is because of the 'keepdims=True' argument used in np.mean()
# np.mean(seq) will apply the mean on the entire array
print(np.mean(seq).shape)
# np.mean(seq, axis=0) will apply the mean along the first dimension, and return a 2D array
print(np.mean(seq, axis=0).shape)
# but the operators will return a filtered sequence with same number of dimensions
print(noise3d.opr.dt(seq).shape)

()
(100, 100)
(1, 100, 100)


In [5]:
# Same goes for dv and dh as dt : the mean is just along other dimension
print("Base operators dt, dv, dh")
print(noise3d.opr.dt(seq).shape)
print(noise3d.opr.dv(seq).shape)
print(noise3d.opr.dh(seq).shape)

Base operators dt, dv, dh
(1, 100, 100)
(100, 1, 100)
(100, 100, 1)


# Extractor operators idt, idv, idh

The idt, idv, and idh operators will simply return the noise of the seq in that dimension.  
For example, `idt` will simply return `seq - dt(seq)`: dt(seq) computes the non-temporal noise, and substracts it to the input sequence. Hence the returned seq is just the temporal-varying signal, ie temporal noise.


In [6]:
# For idt, idv, and idh operators : the returned sequence is just the input sequence,
# from which the filtered sequence is substracted : thanks to numpy's broadcast feature, 
# the returned sequence is not only the same dimension, but also the same shape as input
print("Extract operators idt, idv, idh")
print(noise3d.opr.idt(seq).shape) # this is basicaly the temporal noise of the input sequence
print(noise3d.opr.idv(seq).shape)
print(noise3d.opr.idh(seq).shape)

Extract operators idt, idv, idh
(100, 100, 100)
(100, 100, 100)
(100, 100, 100)


# Noise sequence extraction n_s, n_t, ...
By applying various combination of operators, you can extract each of the 7 noise sequences.

In [7]:
# n_s

# The first, simplest operation is to apply average temporaly, verticaly, and horizontaly. 
# This will return the mean of the input, often denoted "s" : 
print(noise3d.opr.n_s(seq))
print(noise3d.opr.n_s(seq).shape)

[[[-0.11922236]]]
(1, 1, 1)


In [8]:
# n_t, n_v, n_h

# Then you can average along 2 direction, and extract the noise along the 3rd dimension; 
# This results in the 1D noises : t-noise, v-noise, and h-noise sequences 
print(noise3d.opr.n_t(seq).shape)  # pure temporal noise
print(noise3d.opr.n_v(seq).shape)  # pure vertical noise
print(noise3d.opr.n_h(seq).shape)  # pure horizontal noise

(100, 1, 1)
(1, 100, 1)
(1, 1, 100)


In [9]:
# n_tv, n_th, n_vh

# Then you can average along 1 dimension, and extract the noise along the other 2 dimensions;
# This results in the 2D noises : tv-noise, th-noise, and vh-noise
print(noise3d.opr.n_tv(seq).shape)  # pure temporal-vertical noise
print(noise3d.opr.n_th(seq).shape)  # pure temporal-horizontal noise
print(noise3d.opr.n_vh(seq).shape)  # pure vertical-horizontal noise

(100, 100, 1)
(100, 1, 100)
(1, 100, 100)


In [10]:
# n_tvh

# Finaly, you can extract all the 1D and 2D noises of the sequence, leaving only the 3D tvh-noise
print(noise3d.opr.n_tvh(seq).shape)  # pure temporel-vertical-horizontal noise

(100, 100, 100)


All these noise-sequence extraction are wrapped up in a single function. All the noise sequences are returned in a tuple, along with the input sequence.

In [11]:
noise_seqs = noise3d.opr.get_all_3d_noise_seq(seq, names=True)
n_t, n_v, n_h, n_tv, n_th, n_vh, n_tvh, tot, names = noise_seqs

When can then acces the power of each noise component, by taking the variance or standard deviation:

In [12]:
for noise_seq, noise_name in zip(noise_seqs, names):
    print(f"Var of noise {noise_name:3} : {np.var(noise_seq):.3f}")

# Results to compare to the sigmas with which the sequence was created

Var of noise t   : 0.844
Var of noise v   : 0.844
Var of noise h   : 0.929
Var of noise tv  : 0.996
Var of noise th  : 0.963
Var of noise vh  : 1.017
Var of noise tvh : 0.969
Var of noise tot : 6.562


We see here that indeed there is mostly tvh noise, but also that the measurement method is not perfect, since there are non-zero noise power in tv, th, vh noise.

## Faster, equivalent version

A faster version of this extraction exists. It gives almost rigourously identical results.

### Proof almost equivalent

In [16]:
noise_seqs_fast = noise3d.opr.get_all_3d_noise_seq_fast(seq, names=True)

for seq_base, seq_fast in zip(noise_seqs[0:-1], noise_seqs_fast[0:-1]): #exclude names
    print("Almost equal : ", np.allclose(seq_base, seq_fast))
    print("Same variances : ", np.var(seq_base)==np.var(seq_fast))


Almost equal :  True
Same variances :  True
Almost equal :  True
Same variances :  True
Almost equal :  True
Same variances :  False
Almost equal :  True
Same variances :  True
Almost equal :  True
Same variances :  True
Almost equal :  True
Same variances :  True
Almost equal :  False
Same variances :  True
Almost equal :  True
Same variances :  True


## Proof faster

In [15]:
%timeit noise3d.opr.get_all_3d_noise_seq(seq)
%timeit noise3d.opr.get_all_3d_noise_seq_fast(seq)

34.3 ms ± 3.3 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
12 ms ± 1.46 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Other operators for matrix approach
In addition to this first method to compute the noise powers, other methods are available using a matrix approach. The matrix approach methods need 6 different operators, other than the base dt, dv, dh, idt, idv, idh operators. 
They are noted n_dt, n_dv, n_dh, n_dtdv, n_dtdh, n_dvdh, and wrapped up in a single function get_all_3D_mean_seq.
For now just remember that they are similar yet different operators, but serve the same purpose with another approach.

In [17]:
print(noise3d.opr.n_dt(seq).shape) # t-averaged, same as dt(seq)
print(noise3d.opr.n_dt(seq).shape) # v-averaged, same as dv(seq)
print(noise3d.opr.n_dh(seq).shape) # h-averaged, same as dh(seq)

print(noise3d.opr.n_dtdv(seq).shape) # t-and-v-averaged, same as dt(dv(seq))
print(noise3d.opr.n_dtdh(seq).shape) # t-and-h-averaged, same as dt(dh(seq))
print(noise3d.opr.n_dvdh(seq).shape) # t-and-v-averaged, same as dv(dh(seq))

meaned_seq = noise3d.opr.get_all_3D_mean_seq(seq, names=True)
n_dvdh, n_dtdh, n_dtdv, n_dh, n_dv, n_dt, mnames = meaned_seq

# Note that there is no sense in taking the variance of these sequences, 
# they must be "mixed" before falling back on the classic noise sequences.

(1, 100, 100)
(1, 100, 100)
(100, 100, 1)
(1, 1, 100)
(1, 100, 1)
(100, 1, 1)
